In [1]:
from components.model import TechnologyAdoptionModel


def define_and_run_model():
    model = TechnologyAdoptionModel(100, "Ontario")
    for i in range(80):
        model.step()

In [2]:
# 50% of all time is spent here which further breaks down by
#   35% in the actual agents' step and
#   11% in data collection, most of which in "energy_demand_ts"

%timeit define_and_run_model()
# 7.38 s ± 323 ms (after optimisation - 79f10fa)

7.38 s ± 323 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
from batch import BatchResult
batch_parameters = {
    "N": [41],
    "province": ["Ontario"],
    "random_seed": range(20, 25),
    "start_year": 2000,
    "n_segregation_steps": [40],
    "interact": [False],
    # "tech_att_mode_table": [tech_attitude_scenario],
    "ts_step_length": ["W"],
    "hp_subsidy": [0.3]
}
%timeit BatchResult.from_parameters(batch_parameters, display_progress=True, force_rerun=True)
# 17.6 s ± 977 ms (after optimisation - 79f10fa)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

17.6 s ± 977 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [98]:
# from decision_making.mcda import calc_score, normalize
import pandas as pd
import numpy as np
from functools import partial
import numba


# @numba.njit
def normalize(var: np.array, direction=1):
    """normalizes the values to the interval [0, 1].

    Args:
        `var` (np.array): The array to be normalised.
        `direction` (int, optional): indicates wether higher is better (1) or lower is better (-1). Defaults to 1.

    Returns:
        pd.Series: The normalised series.
    """
    # if direction:
    max_ = var.max()
    min_ = var.min()
    if direction == 1:
        norm = (var - min_) / (max_ - min_)
    elif direction == -1:
        norm = (max_ - var) / (max_ - min_)

    return norm

def calc_scores(
    agent,
):
    techs_df = agent.heat_techs_df.loc[agent.model.available_techs, :]
    techs_df["attitude"] = agent.tech_attitudes

    # normalize values
    cost_norm = normalize(techs_df["annual_cost"].values, direction=-1)
    emissions_norm = normalize(techs_df["emissions[kg_CO2/kWh_th]"].values, direction=-1)
    att_norm = normalize(techs_df["attitude"].values)

    np_weights = np.array(list(map(agent.criteria_weights.get,["cost_norm","emissions_norm","attitude_norm"])))
    scores = np.vstack([cost_norm, emissions_norm, att_norm]).T @ np_weights
    return pd.Series(scores, index=techs_df.index)

In [99]:


agent = model.schedule.agents[0]

%timeit -n 1000 calc_scores(agent)
# 6.57 ms ± 707 µs

# after numba normalization:
# 5.47 ms ± 978 µs

# after removal of sorting and numpying calc_score function
# 2.49 ms ± 287 µs

# after removing some dataframe write operations
# 1.11 ms ± 122
# returning the numpy array or the df doesn't appear to make a big difference

# returning techs_df["total_score"] slows things down
# 1.56 ms ± 186 µs

1.13 ms ± 84.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [69]:
techs_df = agent.heat_techs_df.loc[agent.model.available_techs, :]
techs_df["attitude"] = agent.tech_attitudes

# normalize values
cost_norm = normalize(techs_df["annual_cost"].values, direction=-1)
emissions_norm = normalize(techs_df["emissions[kg_CO2/kWh_th]"].values, direction=-1)
att_norm = normalize(techs_df["attitude"].values)

np_weights = np.array(list(map(agent.criteria_weights.get,["cost_norm","emissions_norm","attitude_norm"])))
# techs_df["total_score"] = np.vstack([cost_norm, emissions_norm, att_norm]) @ np_weights

In [97]:
scores = calc_scores(agent)
scores

Gas furnace         0.970432
Oil furnace         0.793876
Biomass furnace     0.059073
Electric furnace    0.901087
Heat pump           0.970620
dtype: float64

In [79]:
score_df = calc_scores(agent)
relevant_cols = filter(lambda x: "norm" in x or "score" in x, score_df.columns)
score_df.loc[:, relevant_cols]

variable,emissions_norm,total_score
Gas furnace,0.704463,0.970432
Oil furnace,0.000000,0.793876
Biomass furnace,0.682828,0.059073
Electric furnace,0.670346,0.901087
Heat pump,1.000000,0.970620


In [63]:
score_df = calc_scores(agent)
relevant_cols = filter(lambda x: "norm" in x or "score" in x, score_df.columns)
score_df.loc[:, relevant_cols]

variable,emissions_norm,attitude_norm,cost_norm,total_score
Gas furnace,0.704463,1.000000,0.995301,0.970432
Oil furnace,0.000000,0.171462,0.920022,0.793876
Biomass furnace,0.682828,0.000000,0.000000,0.059073
Electric furnace,0.670346,0.774030,0.933818,0.901087
Heat pump,1.000000,0.527568,1.000000,0.970620


In [62]:
# %timeit techs_df["attitude_norm"] = normalize(techs_df["attitude"].values)
# 71.4 µs ± 7.63 µs
%timeit normalize(techs_df["attitude"].values)
# 6.44 µs ± 696 ! 10x faster without writing to dataframe

6.44 µs ± 696 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [60]:
# %timeit techs_df.loc[:, ["cost_norm"]] = normalize(techs_df["annual_cost"].values, direction=-1)
# 703 µs ± 66.8 µs 
%timeit normalize(techs_df["annual_cost"].values, direction=-1)
# 6.87 µs ± 505 ns !!! 100x faster without writing to dataframe!

6.87 µs ± 505 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [ ]:
%timeit np.array(list(map(agent.criteria_weights.get,["cost_norm","emissions_norm","attitude_norm"])))

In [57]:
%timeit techs_df[["cost_norm", "emissions_norm", "attitude_norm"]].values @ np_weights

438 µs ± 50.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [126]:
from components.technologies import Fuels,tech_fuel_map
# from components.technologies import 
def energy_demand_ts(model):
    energy_carrier_demand = dict(zip(Fuels, [0] * len(Fuels)))

    zero_demand_fuels = list(Fuels)
    # retrieve the energy demand from each agent
    for a in model.schedule.agents:
        # get fueltype of agent
        fuel = a.heating_tech.fuel
        if fuel in zero_demand_fuels:
            zero_demand_fuels.remove(fuel)
        energy_carrier_demand[fuel] += a.current_fuel_demand.values

    any_demand_fuel = set(Fuels).difference(zero_demand_fuels).pop()
    for fuel in zero_demand_fuels:
        energy_carrier_demand[fuel] = energy_carrier_demand[any_demand_fuel] * 0

    return energy_carrier_demand


# def energy_demand_ts(model):
#     demand_srs = list(map(lambda a: a.current_fuel_demand, model.schedule.agents))
#     demand_df = pd.concat(demand_srs, axis=1)
#     # replace techs with fuels
#     demand_df.columns = [tech_fuel_map[tech] for tech in demand_df.columns]
#     demand_df = demand_df.T.reset_index().groupby("index").sum().T

#     missing_fuels = set(Fuels).difference(demand_df.columns)

#     # add missing fuel
#     demand_df[[*missing_fuels]] = 0
    
#     return {col:demand_df[col] for col in demand_df}